In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# from linearmodels.panel.data import PanelData
# from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt
import pyblp

In [2]:
product_data=pd.read_csv("../../data/merged/len4_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
product_data["market_ids"]=product_data["id"].astype(int).astype(str)+"-"+product_data["year"].astype(str)
product_data.rename(columns={"薬価":"prices","r_share":"shares",
                    "elasped_0":"demand_instruments0",
                    "elasped_1":"demand_instruments1",
                    'elasped_2': 'demand_instruments2',
                    'elasped_3': 'demand_instruments3',
                    # 'elasped_4': 'demand_instruments4',
                    # 'elasped_5': 'demand_instruments5',
                    # 'elasped_6': 'demand_instruments6',
                    # 'elasped_7': 'demand_instruments7',
                    # 'elasped_8': 'demand_instruments8',
                    # 'elasped_9': 'demand_instruments9'
                    # "year":"demand_instruments2",
                    "generic_per":"demand_instruments4",
                    },inplace=True)
product_data=product_data.loc[product_data["shares"]>0]
# data["shares"]=data["r_share"]
product_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,prices,後発品区分,総計,year,in_hospital,...,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0,q_share,shares,market_ids
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0,0.182126,0.182126,1145-2014
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0,0.080302,0.080302,1145-2014
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0,0.066275,0.066275,1145-2014
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0,0.410299,0.410299,1145-2014
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0,0.260997,0.260997,1145-2014


In [3]:
logit_formulation = pyblp.Formulation('prices+oral+in_hospital+後発品区分', absorb='C(id)+C(year)')
logit_formulation

prices + oral + in_hospital + 後発品区分 + Absorb[C(id)] + Absorb[C(year)]

In [4]:
product_data["shares"]-=1e-10
problem = pyblp.Problem(logit_formulation, product_data)
problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N     K1    MD    ED 
---  -----  ----  ----  ----
344  29653   4     8     2  

Formulations:
     Column Indices:          0      1         2         3  
--------------------------  ------  ----  -----------  -----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分


Dimensions:
 T     N     K1    MD    ED 
---  -----  ----  ----  ----
344  29653   4     8     2  

Formulations:
     Column Indices:          0      1         2         3  
--------------------------  ------  ----  -----------  -----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分

In [5]:
logit_results = problem.solve()
logit_results

Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +1.686456E+01     0      +3.370407E+41  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +5.926081E+00     0      +2.982098E+18      +5.480772E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分     
---------------  ---------------  ---------------  ---------------
 -1.509749E-03    +1.522522E+00    +4

Detected that the 2SLS weighting matrix is nearly singular with condition number +2.308148E+18. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +1.930138E+19. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +5.456635E+19. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +1.824844E+19. To disable singularity checks, set options.singular_tol = numpy.inf.


Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +5.926081E+00     0      +2.982098E+18      +5.480772E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分     
---------------  ---------------  ---------------  ---------------
 -1.509749E-03    +1.522522E+00    +4.543447E-01    -2.449966E+00 
(+1.578384E-03)  (+9.525010E-02)  (+4.288114E-02)  (+7.768697E-01)

In [6]:
import datetime

# Get the current date and time
current_time = datetime.datetime.now()

# Format the time as a string
time_string = current_time.strftime("%Y-%m-%d_%H-%M-%S")

logit_results.to_pickle(f"./results/MNL_len4_{time_string}.pkl")